In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json
import base64
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


## JIRA Spring

In [2]:
auth=HTTPBasicAuth("ShengL", "Ls2010506-s")

#### Board

In [3]:
board_url = 'https://jira.spring.io/rest/agile/1.0/board' 

headers = {
   "Accept": "application/json"
}

board = requests.request("GET",board_url,headers=headers,auth=auth).json()

board_df = pd.json_normalize(board['values'])
board_df.head()
board_df.to_csv('jiraSpring_board.csv')  
board_id = board_df['id']


#### Sprint

In [4]:


headers = {
   "Accept": "application/json"
}

payload_sprint = {
'maxResults':200
}

list1 = []
for i in board_id:
   sprint_url = 'https://jira.spring.io/rest/agile/1.0/board/' + f'{i}' + '/sprint'
   sprint = requests.request("GET",sprint_url ,headers=headers,auth=auth, params=payload_sprint).json()
   try:
      list1.append(sprint['values'])
   except:
      pass

In [ ]:
sprint_df = pd.json_normalize(list1)
sprint_result = pd.DataFrame()
sprint_result.head()

""


In [ ]:
sprint_result1 = sprint_df.iloc[1].apply(pd.Series)
sprint_result1['board_id'] = board_df['id'][1]
sprint_result = pd.concat([sprint_result,sprint_result1])
sprint_result.head(2)

,id,self,state,name,startDate,endDate,completeDate,activatedDate,originBoardId,goal,board_id
0,14.0,https://jira.spring.io/rest/agile/1.0/sprint/14,closed,Babbage - RC1 - Stage Two,2013-07-15T13:06:47.325Z,2013-08-05T13:06:00.000Z,2013-08-05T16:06:42.063Z,2013-07-15T13:06:47.325Z,NaN,NaN,27
1,17.0,https://jira.spring.io/rest/agile/1.0/sprint/17,closed,The Road To SpringOne,2013-08-07T10:24:36.975Z,2013-09-06T10:24:36.975Z,2013-09-16T20:12:01.030Z,2013-08-07T10:24:36.975Z,NaN,NaN,27


In [ ]:
for i in sprint_df.index:
   sprint_df_1 = sprint_df.iloc[i].apply(pd.Series)
   sprint_df_1['board_id'] = board_df['id'][i]
   frame = [sprint_result, sprint_df_1]
   sprint_result = pd.concat(frame)

In [ ]:
sprint_result = sprint_result.reset_index()
sprint_result = sprint_result.drop(columns=['index'])
sprint_result.to_csv('jiraSpring_sprint.csv')

NameError: name 'sprint_result' is not defined

#### Issues


In [ ]:
headers = {
   "Accept": "application/json"
}

payload_issues = {
'maxResults':1000
}

list1 = []
for i in board_id:
   issues_url = 'https://jira.spring.io/rest/agile/1.0/board/' + f'{i}' + '/issue'
   issues = requests.request("GET",issues_url ,headers=headers,auth=auth, params=payload_issues).json()
   try:
      list1.append(issues['issues'])
   except:
      pass

In [ ]:
issue_df = pd.json_normalize(list1)

In [ ]:
issue_result = pd.DataFrame()

for i in issue_df.index:
   issue_df_1 = issue_df.iloc[i].apply(pd.Series)
   issue_df_1['board_id'] = board_df['id'][i]
   frame = [issue_result, issue_df_1]
   issue_result = pd.concat(frame)

issue_result = issue_result.reset_index()
issue_result = issue_result.drop(columns=['index'])

issue_result.head(5)

,expand,id,self,key,fields.issuetype.self,fields.issuetype.id,fields.issuetype.description,fields.issuetype.iconUrl,fields.issuetype.name,fields.issuetype.subtask,...,fields.customfield_10781,fields.customfield_10782,fields.customfield_10783,fields.priority,fields.creator,fields.customfield_10000,fields.customfield_10002,fields.customfield_10681,fields.customfield_10682,fields.customfield_10683
0,"operations,versionedRepresentations,editmeta,c...",40607,https://jira.spring.io/rest/agile/1.0/issue/40607,INT-2207,https://jira.spring.io/rest/api/2/issuetype/4,4,An improvement or enhancement to an existing f...,https://jira.spring.io/secure/viewavatar?size=...,Improvement,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"operations,versionedRepresentations,editmeta,c...",40701,https://jira.spring.io/rest/agile/1.0/issue/40701,INT-2228,https://jira.spring.io/rest/api/2/issuetype/4,4,An improvement or enhancement to an existing f...,https://jira.spring.io/secure/viewavatar?size=...,Improvement,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"operations,versionedRepresentations,editmeta,c...",40700,https://jira.spring.io/rest/agile/1.0/issue/40700,INT-2227,https://jira.spring.io/rest/api/2/issuetype/1,1,A problem which impairs or prevents the functi...,https://jira.spring.io/secure/viewavatar?size=...,Bug,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"operations,versionedRepresentations,editmeta,c...",40699,https://jira.spring.io/rest/agile/1.0/issue/40699,INT-2226,https://jira.spring.io/rest/api/2/issuetype/4,4,An improvement or enhancement to an existing f...,https://jira.spring.io/secure/viewavatar?size=...,Improvement,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"operations,versionedRepresentations,editmeta,c...",40693,https://jira.spring.io/rest/agile/1.0/issue/40693,INT-2225,https://jira.spring.io/rest/api/2/issuetype/3,3,A task that needs to be done.,https://jira.spring.io/secure/viewavatar?size=...,Task,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
issue_result.shape

(47000, 203)

In [ ]:
issue_result.to_csv('jiraSpring_issues.csv')